# Database and Exploratory Analysis
# 1. Preparation

## 1.1. Import Library

### 1.1.1. Import Standard Library

In [ ]:
import glob
import sqlite3
import csv
import pandas as pd
import numpy as np

### 1.1.2. Import Plotting Library: Matplotlib

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import FuncFormatter

### 1.1.3. Import Interactive Graphing Library: Bokeh

In [ ]:
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.plotting import figure, output_notebook
from bokeh.models import (ColumnDataSource, LogColorMapper, LinearColorMapper, ColorBar,
SingleIntervalTicker, LogTicker, BasicTicker, NumeralTickFormatter, LinearAxis)
from bokeh.palettes import Viridis256
from bokeh.layouts import gridplot
from bokeh.transform import transform

## 1.2. Set Font Style
Modify font style to display Chinese characters properly

In [ ]:
plt.rcParams['font.family'] = ['Microsoft JhengHei'] 
plt.rcParams['font.serif'] = ['Microsoft JhengHei'] 
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False

# 2. Create Database

## 2.1. Define Function

### 2.1.1. Define Function to Execute Database Commands

In [ ]:
def execute_db(fname, sql_cmd):
    # Establish a connection to SQLite database by creating a connection object "conn."
    conn = sqlite3.connect(fname)
    # Create a cursor object using the cursor method of "conn."
    c = conn.cursor()
    # Call the cursor’s execute method. 
    c.execute(sql_cmd)
    # Save the changes.
    conn.commit()
    # Close the connection.
    conn.close()

### 2.1.2. Define Function to Select Data from Database

In [ ]:
def select_db(fname, sql_cmd):
    # Establish a connection to SQLite database by creating a connection object "conn."
    conn = sqlite3.connect(fname)
    # Create a cursor object using the cursor method of "conn."
    c = conn.cursor()
    # Call the cursor’s execute method. 
    c.execute(sql_cmd)
    # Fetch all rows of a query result.
    rows = c.fetchall()
    # Close the connection.
    conn.close()
    # Return query results. 
    return rows

## 2.2. Create Blank Database

### 2.2.1. Create a Blank Database for Data 1

In [ ]:
# Create a SQLite database file named "db1." 
db_name = 'db1.sqlite'
# Create a table named "Data1."
cmd = ("CREATE TABLE Data1 "
       "(id INTEGER PRIMARY KEY AUTOINCREMENT, "
       "Store TEXT, Order TEXT, "
       "Fruit_Name_ID TEXT, Fruit_Name TEXT, "
       "Qty INTEGER)")
# Execute function "execute_db" with "fname" and "sql_cmd" being specified. 
execute_db(db_name, cmd)

### 2.2.2. Create a Blank Database for Data 2

In [ ]:
# Create a SQLite database file named "db2." 
db_name = 'db2.sqlite'
# Create a table named "Data2."
cmd = ("CREATE TABLE Data2 "
       "(id INTEGER PRIMARY KEY AUTOINCREMENT, "
       "Fruit_Type_ID TEXT, Fruit_Type TEXT)")
# Execute function "execute_db" with "fname" and "sql_cmd" being specified. 
execute_db(db_name, cmd)

## 2.3. Import Data into Database

### 2.3.1. Import Data 1 into Database Table

In [ ]:
# Find all the files in a specific directory. 
files = glob.glob('G:\\Data1\\*.xls')

for file in files:
    # Import data from all the xls files. 
    data = pd.read_excel(file)
    # Remove a column by index.
    data = data.drop(data.columns[[0]], axis = 1)
    # Establish a connection to SQLite database "db1."
    con = sqlite3.connect('db1.sqlite')
    # Write records stored in dataframe "data" to table "Data1" in SQLite database "db1."
    # If table "Data1" already exists, insert new values to the existing table.
    data.to_sql('Data1', con, index = False, if_exists = 'append')
    # Save the changes.
    con.commit()
    # Close the connection.
    con.close()  

### 2.3.2. Import Data 2 into Database Table

In [ ]:
# Import data from xlsx file.  
data = pd.read_excel('G:\\Data2.xlsx')
# Establish a connection to SQLite database "db2."
con = sqlite3.connect('db2.sqlite')
# Write records stored in dataframe "data" to table "Data2" in SQLite database "db2."
# If table "Data2" already exists, insert new values to the existing table.
data.to_sql('Data2', con, index = False, if_exists = 'append')
# Save the changes.
con.commit()
# Close the connection.
con.close()  

# 3. Combine Data from Two Databases

## 3.1. Option 1: Combine Data by SQL

In [ ]:
# Establish a connection to SQLite database "db1."
conn = sqlite3.connect('db1.sqlite')
# Create a cursor object using the cursor method of "conn."
c = conn.cursor()
# Attach database "db2" to "db1."
c = c.execute("ATTACH DATABASE 'db2.sqlite' AS 'db2'")
# Save the changes.
conn.commit()
# Return all records from the left table "Data1" in "db1" aliased as "P1"
# and the matched records from the right table "Data2" in "db2" aliased as "P2"
# on the condition that the first 4 letters of records of "Fruit_Name_ID" in "P1" 
# are identical with records of "Fruit_Type_ID" in "P2."
outcome = pd.read_sql_query('''
                            SELECT *
                            FROM main.Data1 P1
                            LEFT JOIN
                            db2.Data2 P2
                            ON SUBSTR (P1.Fruit_Name_ID, 1 ,4) = P2.Fruit_Type_ID;
                            ''', conn)
# Close the connection.
conn.close() 

## 3.2. Option 2: Combine Data by Python

In [ ]:
# Establish a connection to SQLite database "db1."
conn_data1 = sqlite3.connect('db1.sqlite')
# Create a cursor object using the cursor method of "conn_data1."
cursor_data1 = conn_data1.cursor()
# Save the changes.
conn_data1.commit()
# Return all records from table "Data1" in "db1." 
data1 = pd.read_sql_query('''
                          SELECT *
                          FROM main.Data1
                          ''', conn_data1)
# Close the connection.
conn_data1.close()

In [ ]:
# Get the first 4 letters of the string for all records in "Fruit_Name_ID" 
# and name the column of returned values as "Fruit_Type_ID."
data1['Fruit_Type_ID'] = data1['Fruit_Name_ID'].str.slice(0,4)

In [ ]:
# Establish a connection to SQLite database "db2."
conn_data2 = sqlite3.connect('db2.sqlite')
# Create a cursor object using the cursor method of "conn_data2."
cursor_data2 = conn_data2.cursor()
# Save the changes.
conn_data2.commit()
# Return all records from table "Data2" in "db2." 
data2 = pd.read_sql_query('''
                          SELECT *
                          FROM main.Data2
                          ''', conn_data2)
# Close the connection.
conn_data2.close()

In [ ]:
# Return all records from the left table "Data1" in "db1"
# and the matched records from the right table "Data2"
# on the condition that the two tables share the same "Fruit_Type_ID." 
outcome = data1.merge(data2, left_on='Fruit_Type_ID', right_on='Fruit_Type_ID', how='left')

In [ ]:
# Remove columns by coulmn names.
outcome = outcome.drop('id_x', axis = 1)
outcome = outcome.drop('id_y', axis = 1)

# 4. Exploratory Data Analysis

In [ ]:
# Copy "outcome" for further analysis to prevent data corruption.
eda1 = outcome.copy()

In [ ]:
# 1st Groupby: 
# Split data into groups by "Store" and "Fruit_Type" to count frequencies of unique values. 
eda2 = eda1.groupby(['Store','Fruit_Type']).size().reset_index()
eda2.head()

In [ ]:
# Remove the 3rd column which is of no use to further analysis
eda3 = eda2.drop(eda2.columns[2], axis = 1)
eda3.head()

In [ ]:
# 2nd Groupby:
# Split data into groups by "Store" and join "Fruit_Type" by "," for each row of records.
eda4 = eda3.groupby('Store')['Fruit_Type'].apply(','.join).reset_index()
eda4.head()

In [ ]:
# 3rd Groupby:
# Split data into groups by "Fruit_Type" and count frequencies of unique values.
eda5 = eda4.groupby('Fruit_Type').size().reset_index()
eda5.head()

In [ ]:
# Name the column of returned values as "Count."
eda5.columns = ['Fruit', 'Count']
eda5.head()

In [ ]:
# Export analysis result to csv file.
eda5.to_csv("G:\\Analysis.csv", index = False, encoding = "ansi")

# 5. Data Visualization

## 5.1. Import Data for Plotting

In [ ]:
# Import data from csv file
# and assign the result to an object named "df1."
df1 = pd.read_csv("G:\\Analysis.csv", encoding = 'ansi')
df1.head()

In [ ]:
# Count the number of comma in column "Fruit_Type" for each string 
# and assign the result to an object named "df2."
df2 = df1['Fruit_Type'].str.count(',').to_frame()
# Change column name to "Comma"
df2.columns = ['Comma']

In [ ]:
# Add column "Comma" to df1 
# and assign the result to an object named "df3."
df3 = df1.join(df2)
df3.head()

In [ ]:
# Get unique values in column "Comma." 
commas = list(df3.groupby('Comma').size().to_frame().reset_index()['Comma']) 
print(commas)
# Count the number of unique values in column "Comma" 
# to determine the number of figures to be displayed.
df3.groupby('Comma').size().count()

In [ ]:
# Create an empty dataframe named df4.
df4 = pd.DataFrame() 

In [ ]:
# For each unique value in column "Comma," 
# sort df3 in descending order by column "Count" from which top 20 elements are to be selected
# and added to df4. 
for comma in commas:
    temp = df3[df3['Comma'] == comma].sort_values(['Count'], ascending = False)[0:20]
    df4 = df4.append(temp)
df4.head()

## 5.2. Plot Static Bar Chart by Matplotlib

In [ ]:
# Create 9 subplots composed of 3 rows with 3 graphs in a row
# Though there are 24 unique values in column "Comma" and therefore 24 subplots are to be expected,
# the excluded are those have only one record and thus are not well suited to further visualization. 
fig, axs = plt.subplots(figsize=(20,10), 
                        nrows=3, ncols=3,   
                        gridspec_kw = dict(wspace=0.3, hspace=0.3))

In [ ]:
# Get unique values in column "Comma." 
grouped = df4.groupby('Comma')

In [ ]:
# Set the colorbar legend.
cmap = plt.cm.rainbow

In [ ]:
# Make an iterator named "target" aggregating elements from
# keys of "grouped" which are the number of commas and
# flattened 3*3 array of axes to plot to all subplots in a single loop.
targets = zip(grouped.groups.keys(), axs.flatten())

In [ ]:
for i, (key, ax) in enumerate(targets):

    # When the number of commas is greater than 8, 
    # there exists less than 20 records and hence should be excluded from further visualization. 
    if key > 8:
    break
    
    # When the number of commas is less than 8, do as follows.
    # Set the limits of colorbar by corresponding number of "Count."        
    norm = matplotlib.colors.Normalize(vmin=grouped.get_group(key)['Count'].values.min(), 
                                       vmax=grouped.get_group(key)['Count'].values.max())
    
    # Specify which color is to be applied to corresponding number of "Count."
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    
    # Create a vertical bar chart with X-axis showing "Fruit_Type" whose number of "Count" 
    # is among top 20 and with Y-axis having range from 0 to the maximum of "Count" 
    # to which the colorbar applies. 
    ax.bar(grouped.get_group(key)['Fruit_Type'], grouped.get_group(key)['Count'], 
           color = cmap(norm(grouped.get_group(key)['Count'].values)))
    
    # Set the tilte of the subplot.
    ax.set_title('分組%d'% (key+1), fontsize=16)
    
    # Set the label of Y-axis.
    ax.set_ylabel('計數', fontsize=16)
    
    # When the number of commas is less than 6,
    # which suggests that the record of "Fruit Type" contains 7 kinds of fruit,
    # add thousand separator to tick labels of the colorbar which are alighed with Y-axis.    
    if key < 6:
        comma_fmt = FuncFormatter(lambda x, p: format(int(x), ','))
        fig.colorbar(sm, ax=ax, ticks=ax.get_yticks(), format=comma_fmt)
    
    # When the number of commas is greater than or equal to 6,
    # which suggests that the record of "Fruit Type" contains no less than 7 kinds of fruit,
    # align tick labels of the colorbar with Y-axis formated without thousand separator.
    else:
        fig.colorbar(sm, ax=ax, ticks=ax.get_yticks())
    
    # Since records of "Fruit Type" tend to be long strings 
    # and are thus likely to overlap one another, 
    # it is better to make X-axis invisible. 
    ax.xaxis.set_visible(False)
    
    # Add thousand separator to labels of Y-axis.
    ax.get_yaxis().set_major_formatter(
    tkr.FuncFormatter(lambda x, p: format(int(x), ',')))
   
    # Set the font size of Y-axis labels.
    ax.tick_params(axis='y', which='major', labelsize=12)    
    
plt.show()       

## 5.3. Plot Interactive Bar Chart by Bokeh

In [ ]:
# Get unique values in column "Comma." 
grouped = df4.groupby('Comma')

In [ ]:
# Create an empty list to store plots to be arranged in a grid. 
g = []

In [ ]:
# Get keys of "grouped" which are the number of commas.
targets = grouped.groups.keys()

In [ ]:
# Configure an output to a standalone HTML file.
output_file('5_3_2_InteractiveBarChart.html')

In [ ]:
for i, key in enumerate(targets):

    # When the number of commas is greater than 8, 
    # there exists less than 20 records and hence should be excluded from further visualization. 
    if key > 8:
        break

    # Get top 20 by column "Count" and their "Fruit_Type" and the number of "Count" accordingly.
    type_list = list(grouped.get_group(key)['Fruit_Type'])[:20]
    count_list = list(grouped.get_group(key)['Count'])[:20]
    
    # Specify the records to be plotted
    source = ColumnDataSource(data={
        'types': type_list,
        'counts': count_list,
    })
    
    # Set the colorbar legend with limits determined by corresponding number of "Count"  
    mapper = LinearColorMapper(palette=Viridis256, 
                               low=grouped.get_group(key)['Count'].values.min(), 
                               high=grouped.get_group(key)['Count'].values.max())

    # Set overall formats of the subplot 
    # among which the most important is to specify the display of tooltips.
    p = figure(x_range=type_list, plot_height=10, plot_width=110, title='分組%d'% (key+1),
               toolbar_location=None, tools="", tooltips=[('類型', '@types')])
    
    # Create a vertical bar chart with X-axis showing "Fruit Type" whose number of "Count" 
    # is among top 20 and with bar height reflecting the number of "Count."
    p.vbar(x='types', top='counts', width=0.9, source=source, fill_color=transform('counts', mapper), 
           line_color="white", line_width=0.05)
    
    # When the number of commas is less than 6,
    # which suggests that the record of "Fruit Type" contains 7 kinds of fruit,
    # add thousand separator to tick labels of the colorbar.  
    if key < 6:
        color_bar = ColorBar(color_mapper=mapper, location=(2, 2),
                             ticker=BasicTicker(desired_num_ticks=10),
                             formatter=NumeralTickFormatter(format='0,0'))
        
    # When the number of commas is greater than or equal to 6,
    # which suggests that the record of "Fruit Type" contains no less than 7 kinds of fruit,
    # add decimal points to tick labels of the colorbar. 
    else:
        color_bar = ColorBar(color_mapper=mapper, location=(2, 2),
                             ticker=BasicTicker(desired_num_ticks=10),
                             formatter=NumeralTickFormatter(format='0.0[00]'))
    
    # Set the position of the colorbar.
    p.add_layout(color_bar, 'right')

    # Make X-grid invisible.
    p.xgrid.grid_line_color = None
    
    # Make X-axis invisible.
    p.xaxis.visible = False
    
    # Start the vertical scale at 0.
    p.y_range.start = 0
    
    # When the number of commas is less than 6,
    # which suggests that the record of "Fruit Type" contains 7 kinds of fruit,
    # add thousand separator to labels of Y-axis. 
    if key < 6:
        p.yaxis.formatter = NumeralTickFormatter(format='0,0')
    
    # When the number of commas is greater than or equal to 6,
    # which suggests that the record of "Fruit Type" contains no less than 7 kinds of fruit,
    # add decimal points to labels of Y-axis.  
    else:
        p.yaxis.formatter = NumeralTickFormatter(format='0.0[00]')        
     
    # Add the subplot to list "g."
    g.append(p)

# Create a grid where all of the subplots will be placed.
grid = gridplot(g, ncols=3, plot_width=420, plot_height=420) 
show(grid)